![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [194]:
#librerias necesarias
import pandas as pd
import numpy as np

bank_marketing_df= pd.read_csv("bank_marketing.csv")

In [195]:
#explorando dataframe
bank_marketing_df.head(10)

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no
5,5,45,services,married,basic.9y,unknown,no,may,5,198,1,0,nonexistent,93.994,4.857,no
6,6,59,admin.,married,professional.course,no,no,may,3,139,1,0,nonexistent,93.994,4.857,no
7,7,41,blue-collar,married,unknown,unknown,no,may,12,217,1,0,nonexistent,93.994,4.857,no
8,8,24,technician,single,professional.course,no,yes,may,21,380,1,0,nonexistent,93.994,4.857,no
9,9,25,services,single,high.school,no,yes,may,5,50,1,0,nonexistent,93.994,4.857,no


In [196]:
#verificando Dataframe
bank_marketing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

## cleaning requirements

In [197]:
bank_marketing_df["job"]= bank_marketing_df["job"].str.replace(".", "_")

In [198]:
bank_marketing_df["education"] = bank_marketing_df["education"].str.replace(".", "_")
bank_marketing_df["education"] = bank_marketing_df["education"].replace('unknown' , np.nan)


In [199]:
bank_marketing_df["mortgage"].unique()

array(['no', 'yes', 'unknown'], dtype=object)

In [200]:
bank_marketing_df["credit_default"].unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [201]:
bank_marketing_df["credit_default"]=bank_marketing_df["credit_default"].replace({'yes':True, 'no':False, 'unknown':False})
bank_marketing_df["mortgage"]=bank_marketing_df["mortgage"].replace({'yes':True, 'no':False, 'unknown':False})

In [202]:
bank_marketing_df['previous_outcome'].unique()

array(['nonexistent', 'failure', 'success'], dtype=object)

In [203]:
# Pasando de object a booleano

bank_marketing_df["previous_outcome"]=bank_marketing_df["previous_outcome"].replace({'success':True, 'nonexistent':False, 'success':False}).astype(bool)

In [204]:
bank_marketing_df['campaign_outcome'].unique()

array(['no', 'yes'], dtype=object)

In [205]:
# Pasando de object a booleano

bank_marketing_df['campaign_outcome']=bank_marketing_df['campaign_outcome'].replace({'yes':True, 'no':False}).astype(bool)

In [206]:
bank_marketing_df['year']=2022

In [207]:
# Create a dictionary to map month names to numeric values
month_dict = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05', 'jun': '06',
              'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'}

# Convert the month column to lowercase and map the month names to numeric values
bank_marketing_df['month'] = bank_marketing_df['month'].str.lower().map(month_dict)

# Convert the year, month, and day columns to datetime format
bank_marketing_df['last_contact_date'] = pd.to_datetime(bank_marketing_df[['year', 'month', 'day']], format='%Y-%m-%d')


bank_marketing_df['last_contact_date'].dtype

dtype('<M8[ns]')

In [208]:
bank_marketing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   age                         41188 non-null  int64         
 2   job                         41188 non-null  object        
 3   marital                     41188 non-null  object        
 4   education                   39457 non-null  object        
 5   credit_default              41188 non-null  bool          
 6   mortgage                    41188 non-null  bool          
 7   month                       41188 non-null  object        
 8   day                         41188 non-null  int64         
 9   contact_duration            41188 non-null  int64         
 10  number_contacts             41188 non-null  int64         
 11  previous_campaign_contacts  41188 non-null  int64     

In [209]:
bank_marketing_df.head(5)

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,year,last_contact_date
0,0,56,housemaid,married,basic_4y,False,False,05,13,261,1,0,False,93.994,4.857,False,2022,2022-05-13
1,1,57,services,married,high_school,False,False,05,19,149,1,0,False,93.994,4.857,False,2022,2022-05-19
2,2,37,services,married,high_school,False,True,05,23,226,1,0,False,93.994,4.857,False,2022,2022-05-23
3,3,40,admin_,married,basic_6y,False,False,05,27,151,1,0,False,93.994,4.857,False,2022,2022-05-27
4,4,56,services,married,high_school,False,False,05,3,307,1,0,False,93.994,4.857,False,2022,2022-05-03


In [210]:
#Creando Dataframe solicitados
client = bank_marketing_df[['client_id','age','job','marital','education','credit_default','mortgage']]

campaign = bank_marketing_df[['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome','last_contact_date']]

economics = bank_marketing_df[['client_id','cons_price_idx','euribor_three_months']]

In [211]:
#client.csv
bank_marketing_df.to_csv('client.csv', index=False, columns = client)

#campaign.csv
bank_marketing_df.to_csv('campaign.csv', index=False, columns = campaign)

#economics.csv
bank_marketing_df.to_csv('economics.csv', index=False, columns = economics)